<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/phunc20/python/blob/master/math/e/e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

In [3]:
def a(n):
    return (1 + 1/n)**n

## How Python Does Exponentiation under The Hood?
Surely it's not doing multiplications the number of power of times?
Maybe it uses logarithms
$$x^n = \exp\left(\ln x^n\right) = \exp\left( n\ln x\right)\,\text{?}$$


Perhaps the `dis` package's analysis of the bytecode could shed some
light on this?

In [4]:
import dis

In [5]:
dis.dis(a)

  2           0 LOAD_CONST               1 (1)
              2 LOAD_CONST               1 (1)
              4 LOAD_FAST                0 (n)
              6 BINARY_TRUE_DIVIDE
              8 BINARY_ADD
             10 LOAD_FAST                0 (n)
             12 BINARY_POWER
             14 RETURN_VALUE


Well... not so much. At least not for someone who knows little
about `BINARY_POWER`...

Cf. the function `float_pow(PyObject *v, PyObject *w, PyObject *z)` in `cpython/Objects/floatobject.c` or <https://github.com/python/cpython/blob/main/Objects/floatobject.c#L743>

```C
    /* Now iv and iw are finite, iw is nonzero, and iv is
     * positive and not equal to 1.0.  We finally allow
     * the platform pow to step in and do the rest.
     */
    errno = 0;
    ix = pow(iv, iw);
    _Py_ADJUST_ERANGE1(ix);
    if (negate_result)
        ix = -ix;
```

In [19]:
import math
from tqdm.notebook import tqdm

In [20]:
MAX = 15
for p in range(1, MAX):
    n = 10**p
    an = a(n)
    head = f"a(10**{p:2d}) = {an}"
    tail = " > e" if an > math.e else " < e"
    print(head + tail)

a(10** 1) = 2.5937424601000023 < e
a(10** 2) = 2.7048138294215285 < e
a(10** 3) = 2.7169239322355936 < e
a(10** 4) = 2.7181459268249255 < e
a(10** 5) = 2.7182682371922975 < e
a(10** 6) = 2.7182804690957534 < e
a(10** 7) = 2.7182816941320818 < e
a(10** 8) = 2.7182817983473577 < e
a(10** 9) = 2.7182820520115603 > e
a(10**10) = 2.7182820532347876 > e
a(10**11) = 2.71828205335711 > e
a(10**12) = 2.7185234960372378 > e
a(10**13) = 2.716110034086901 < e
a(10**14) = 2.716110034087023 < e


In [19]:
1 + 1 / (10**14) == 1

False

In [7]:
1 + 1/(10**14) < 1 + 1/(10**12)

True

In [9]:
from decimal import Decimal

In [13]:
Decimal(1) + Decimal(1)/Decimal(10**14) > 1 + 1/(10**14)

True

In [17]:
for n in range(1, 20):
    if Decimal(1) + Decimal(1)/Decimal(10**n) > 1 + 1/(10**n):
        print(f"{n}: down")
    else:
        print(f"{n}: up")

1: up
2: up
3: down
4: down
5: up
6: down
7: up
8: down
9: up
10: up
11: up
12: up
13: down
14: down
15: up
16: down
17: down
18: down
19: down


In [29]:
n = 16
a(10**n) > math.e

False

In [20]:
1 + 1 / (10**100) == 1

True

In [24]:
enough = 1
n_counter_examples = 0
for n in tqdm(range(10**8, 10**9)):
    this = a(n)
    if this > math.e:
        print(f"a({n}) = {this} > e")
        n_counter_examples += 1
    if n_counter_examples >= enough:
        break

  0%|          | 0/900000000 [00:00<?, ?it/s]

a(100000006) = 2.7182818407282965 > e


In [25]:
a(100000006) > a(10**9)

False

**(?)** $a_{n}$ an increasing sequence?

In [9]:
prev = 0
enough = 5
n_counter_examples = 0
for n in range(1, 10**9):
    this = a(n)
    if this < prev:
        print(f"a({n}) = {this} < a({n-1}) = {prev}")
        n_counter_examples += 1
    if n_counter_examples >= enough:
        break
    prev = this

a(132361) = 2.718271560051147 < a(132360) = 2.718271560052231
a(132918) = 2.7182716030808325 < a(132917) = 2.718271603081553
a(133631) = 2.7182716576400137 < a(133630) = 2.718271657641679
a(134043) = 2.71827168890323 < a(134042) = 2.7182716889039806
a(135059) = 2.7182717651779056 < a(135058) = 2.718271765179464


**(?)** Is `a(n)` bounded? Could we tell `max(Z, key=a)` in advance?

## 

### (?) Is `a` catastrophic or numerically unstable?

In [26]:
math.log(math.e)

1.0

In [27]:
def another(n):
    return math.e**(n * math.log(1+1/n))

In [29]:
MAX = 15
for p in range(1, MAX):
    n = 10**p
    an = a(n)
    ann = another(n)
    if ann != an:  # Use machine epsilon instead
        print(f"{n = }")

n = 10
n = 100000
n = 10000000
n = 100000000
n = 10000000000
n = 10000000000000
n = 100000000000000


In [28]:
MAX = 15
for p in range(1, MAX):
    n = 10**p
    an = another(n)
    head = f"a(10**{p:2d}) = {an}"
    tail = " > e" if an > math.e else " < e"
    print(head + tail)

a(10** 1) = 2.593742460100002 < e
a(10** 2) = 2.7048138294215285 < e
a(10** 3) = 2.7169239322355936 < e
a(10** 4) = 2.7181459268249255 < e
a(10** 5) = 2.718268237192297 < e
a(10** 6) = 2.7182804690957534 < e
a(10** 7) = 2.7182816941320813 < e
a(10** 8) = 2.7182817983473573 < e
a(10** 9) = 2.7182820520115603 > e
a(10**10) = 2.718282053234787 > e
a(10**11) = 2.71828205335711 > e
a(10**12) = 2.7185234960372378 > e
a(10**13) = 2.7161100340869004 < e
a(10**14) = 2.7161100340870226 < e


事後諸葛亮
> The worse part in `a` is `1 + 1/n`.  
> And this is not remedized in `another`,
> so we should not expect `another` to do any
> better than `a`.

### Seeking for Another Approximation Formula

In [34]:
from functools import cache

In [44]:
@cache
def factorial(n):
    if n == 0:
        return 1
    else:
        return factorial(n-1)*n

@cache
def yet_another(n):
    if n == 0:
        return 1
    else:
        return yet_another(n-1) + 1/factorial(n)

In [48]:
math.log10(10**4)

4.0

In [ ]:
MAX = 15
for n in tqdm(range(1, 10**MAX)):
    an = yet_another(n)
    head = f"a({n:15d}) = {an}"
    tail = " > e" if an > math.e else " < e"
    power = math.log10(n)
    if int(power) == power:
        print(head + tail)

  0%|          | 0/999999999999999 [00:00<?, ?it/s]

a(              1) = 2.0 < e
a(             10) = 2.7182818011463845 < e
a(            100) = 2.7182818284590455 > e
a(           1000) = 2.7182818284590455 > e
a(          10000) = 2.7182818284590455 > e


In [45]:
MAX = 15
for p in range(1, MAX):
    n = 10**p
    an = yet_another(n)
    head = f"a(10**{p:2d}) = {an}"
    tail = " > e" if an > math.e else " < e"
    print(head + tail)

a(10** 1) = 2.7182818011463845 < e
a(10** 2) = 2.7182818284590455 > e
a(10** 3) = 2.7182818284590455 > e


RecursionError: maximum recursion depth exceeded